In [74]:
# Artificial Intellegence HW1 
# Jan 20, 2023
# Booth 32200
# Nic Marlton

### Psuedo Code
# dependencies
# dictionary of spot ids (pulled mannually from surfline.com)
# use spot ids list to query surfline api
    # note: either the conditions url or the wave url could be used
    # `conditions` returns ratings 'POOR' to 'EPIC', whereas `wave`
    # returns an optimal rating 0, 1, 2. I chose the optimal rating
    # given its relative simplicity
    # helpful overview of api below:
    # https://pkg.go.dev/github.com/mhelmetag/surflinef/v2#section-readme
# assign surfline ratings to photos
    # in a future itteration of this tool, I would want to pull photos
    # and ratings in real time. that would make this step unnecessary. 
    # for the purposes of this proof of concept, I am mannually assigning 
    # ratings. 
# sort 90% of photos into 4 X 4 - good v bad X crowded v quiet
# store remaining 10 % for test data

### Process after this point occurs in Google's Teachable Machine 

In [75]:
# Dependencies
import pandas as pd
import numpy as np
import requests

In [76]:
# Dictionary of spotIds 
spot_id_dict = {
    'PlesurePoint':'5842041f4e65fad6a7708807',
    'UpperTrestles':'5842041f4e65fad6a7708887',
    'SanClementeStateBeach':'5842041f4e65fad6a77088cf',
    'LowerTrestles':'5842041f4e65fad6a770888a',
    'SteamerLane':'5842041f4e65fad6a7708805',
    'TStreetOverview':'5842041f4e65fad6a7708830',
    'Church':'5842041f4e65fad6a770888b',
    'Strands':'5842041f4e65fad6a77088d5',
    'Jacks':'5842041f4e65fad6a770880b',
    'SaltCreekOverview':'5842041f4e65fad6a770882e',
    # 'Hightower':'584204214e65fad6a7709cbd', // excluding Hightower given it is in FL 
    'Lowers':'5842041f4e65fad6a770888a',
    # 'VenicePierSouthside':'5842041f4e65fad6a7708b17' // excluding Venice Pier given it is in FL
}

In [77]:
# Concatenate data to create list of files w & w/o surfers 
# I did not include these csv files in the git repo because of their size.
# they can be downlaoded with the photo data here:
# 
df1 = pd.read_csv('test_annotations.csv') # csv describing Roboflow test data
df2 = pd.read_csv('train_annotations.csv') # csv describing Roboflow train data 
df3 = pd.read_csv('valid_annotations.csv') # csv describing Roboflow validation data
annotations_df = pd.concat([df1,df2,df3],ignore_index=True)
df_mask = annotations_df['filename'].isna()
annotations_df = annotations_df[~df_mask]
annotations_df = annotations_df['filename'].drop_duplicates().reset_index()
annotations_df = annotations_df['filename']
annotations_df.head()


0    Strands_2021_02_19_0122pm_frame_29_left_jpg.rf...
1    SanClementeStateBeach_2021_02_19_1104am_frame_...
2    PlesurePoint_2021_02_19_1147am_frame_4_right_j...
3    UpperTrestles_2021_02_19_1103am_frame_21_right...
4    SaltCreekOverview_2021_02_19_1121am_frame_30_l...
Name: filename, dtype: object

In [82]:

def get_conditions(spot_id):
    '''
    API Query
    '''
    login_base_url = 'https://services.surfline.com/trusted/token'
    base_url = 'https://services.surfline.com/kbyg/regions/forecasts/conditions'
    # subregion_id = '5842041f4e65fad6a7708887' #'5842041f4e65fad6a7708890' #'58581a836630e24c44878fd4' #'244b3431f9ff2b5dbcadd16ad7f40c9e' #str('617d74bcfea95ce3f0e875bc7aae2fd1'[:24]) #
    response = requests.request("GET", base_url + '?spotId=' + spot_id)
    # print(response.json())
    df = pd.json_normalize(response.json()['data'], record_path =['conditions'])
    return df.head(2)

fcst_df = pd.DataFrame()
for i in spot_id_dict:
    fcst_df = pd.concat([fcst_df,get_conditions(spot_id_dict[i])])

In [84]:
# len('58581a836630e24c44878fd4')
# print(len('5842041f4e65fad6a7708890'))
fcst_df.head().T

,0,1,0,1,0
timestamp,1674201600,1674288000,1674201600,1674288000,1674201600
forecastDay,2023-01-20,2023-01-21,2023-01-20,2023-01-21,2023-01-20
human,True,True,True,True,True
observation,Groomed offshore wind continues but note the d...,Quality conditions continue. Dropping tide mid...,Still some modest NW swell running and the win...,Mix of modest NW swell and selective long-peri...,Still some modest NW swell running and the win...
utcOffset,-8,-8,-8,-8,-8
forecaster.name,Keaton Browning,Keaton Browning,Schaler Perry,Schaler Perry,Schaler Perry
forecaster.avatar,https://www.gravatar.com/avatar/a7628271b83f93...,https://www.gravatar.com/avatar/a7628271b83f93...,https://www.gravatar.com/avatar/ea1e9a0c570c61...,https://www.gravatar.com/avatar/ea1e9a0c570c61...,https://www.gravatar.com/avatar/ea1e9a0c570c61...
am.timestamp,1674223200,1674309600,1674223200,1674309600,1674223200
am.observation,Moderate mid period waves from the WNW. Light ...,Fun sized mid period waves from the WNW. Light...,Small very short period waves from the WNW. Li...,Small mid period waves from the SSW. Light ENE...,Small very short period waves from the WNW. Li...
am.rating,FAIR,FAIR_TO_GOOD,FAIR,FAIR,FAIR
